In [1]:
import os
import json
import pandas as pd
import numpy as np
train = pd.read_csv('transactions111.csv')
test = pd.read_csv('transactions222.csv')

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9643012 entries, 0 to 9643011
Data columns (total 35 columns):
Unnamed: 0                int64
duration                  int64
feature_1                 float64
feature_2                 float64
feature_3                 int64
feature_4                 float64
feature_5                 float64
movie                     int64
multipart_movie           int64
series                    int64
P                         int64
R                         int64
S                         int64
wt_user_mean              float64
per_user_mean             float64
wt_element_mean           float64
per_elemnt_mean           float64
quantity_user_views       int64
quantity_element_views    int64
PPP                       int64
PR                        int64
PRS                       int64
PS                        int64
RRR                       int64
RS                        int64
SSS                       int64
ts-f1                     float64
inter

In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622631 entries, 0 to 1622630
Data columns (total 21 columns):
Unnamed: 0                1622631 non-null int64
user_uid                  1622631 non-null int64
element_uid               1622631 non-null int64
availability              1622631 non-null object
duration                  1622631 non-null int64
feature_1                 1622631 non-null float64
feature_2                 1622631 non-null float64
feature_3                 1622631 non-null int64
feature_4                 1622631 non-null float64
feature_5                 1622631 non-null float64
type                      1622631 non-null object
ts-f1                     1622631 non-null float64
wt_user_mean              1622631 non-null float64
quantity_user_views       1622631 non-null int64
per_user_mean             1622631 non-null float64
wt_element_mean           1622631 non-null float64
quantity_element_views    1622631 non-null int64
per_elemnt_mean           1622631 no

In [6]:
train=train.drop(['Unnamed: 0', 'P', 'R', 'S', 'PPP', 'PR', 'PRS','PS', 'RRR','RS','SSS','a','b','c','d','e','f','aa'],1)
test=test.drop(['Unnamed: 0'],1)
test.isnull().sum()/test.shape[0]

user_uid                  0.0
element_uid               0.0
availability              0.0
duration                  0.0
feature_1                 0.0
feature_2                 0.0
feature_3                 0.0
feature_4                 0.0
feature_5                 0.0
type                      0.0
ts-f1                     0.0
wt_user_mean              0.0
quantity_user_views       0.0
per_user_mean             0.0
wt_element_mean           0.0
quantity_element_views    0.0
per_elemnt_mean           0.0
movie                     0.0
multipart_movie           0.0
series                    0.0
dtype: float64

In [7]:


import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

X_train=train
X_train=X_train.drop(['interaction'],1)

y_train=train.interaction 


estimator=xgb.XGBClassifier(n_estimators=700, max_depth=8, n_jobs=-1 )
estimator.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=700,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622631 entries, 0 to 1622630
Data columns (total 16 columns):
duration                  1622631 non-null int64
feature_1                 1622631 non-null float64
feature_2                 1622631 non-null float64
feature_3                 1622631 non-null int64
feature_4                 1622631 non-null float64
feature_5                 1622631 non-null float64
movie                     1622631 non-null int64
multipart_movie           1622631 non-null int64
series                    1622631 non-null int64
wt_user_mean              1622631 non-null float64
per_user_mean             1622631 non-null float64
wt_element_mean           1622631 non-null float64
per_elemnt_mean           1622631 non-null float64
quantity_user_views       1622631 non-null int64
quantity_element_views    1622631 non-null int64
ts-f1                     1622631 non-null float64
dtypes: float64(9), int64(7)
memory usage: 198.1 MB


In [11]:
X_test=test[['duration',
'feature_1',
'feature_2',
'feature_3',
'feature_4',
'feature_5',
'movie',
'multipart_movie',
'series',
'wt_user_mean',           
'per_user_mean',
'wt_element_mean',
'per_elemnt_mean',  
'quantity_user_views', 
'quantity_element_views',
'ts-f1'  ]]


In [16]:

submit=estimator.predict_proba(X_test)

In [17]:
submit=pd.DataFrame(submit)
bam=test[['user_uid','element_uid']]

bam=bam.reset_index()
bam=bam.drop(['index'], 1)
OLD=pd.concat([bam, submit], 1)
OLD.head()

,user_uid,element_uid,0,1
0,0,7079,0.164395,0.835605
1,131072,7079,0.114669,0.885331
2,393222,7079,0.451093,0.548907
3,262154,7079,0.157323,0.842677
4,13,7079,0.494458,0.505542


In [18]:
OLD['rating']=OLD[1]
OLD=OLD.drop([0, 1],1)
OLD=OLD.sort_values(by=['user_uid', 'rating'], ascending=False)

result={}
for user_uid in pd.unique(OLD.user_uid):
    result[(user_uid).astype(str)] = OLD.element_uid[OLD.user_uid == user_uid].values.tolist()[:20]
    
with open('okkoend.json', 'w') as f:
    json.dump(result, f)
